In [2]:
# Import dependencies
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

In [5]:
# Importing Data Sets 
demographics = pd.read_csv('./Resources/NYC_Demographics.csv')
evictions = pd.read_csv('./Resources/NYC_Evictions.csv')

In [6]:
demographics.head()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,44,100,20,0.45,24,0.55,0,0,44,100
1,10002,35,19,0.54,16,0.46,0,0,35,100,...,35,100,2,0.06,33,0.94,0,0,35,100
2,10003,1,1,1.00,0,0.00,0,0,1,100,...,1,100,0,0.00,1,1.00,0,0,1,100
3,10004,0,0,0.00,0,0.00,0,0,0,0,...,0,0,0,0.00,0,0.00,0,0,0,0
4,10005,2,2,1.00,0,0.00,0,0,2,100,...,2,100,0,0.00,2,1.00,0,0,2,100


In [7]:
evictions.head()

,Court Index Number,Docket Number,Eviction Address,Eviction Apartment Number,Executed Date,Marshal First Name,Marshal Last Name,Residential/Commercial,BOROUGH,Eviction Zip,Ejectment,Eviction/Legal Possession,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,56037/17,339568,547 EAST 168TH STREET,3H,2/26/18,Thomas,Bia,Residential,BRONX,10456,Not an Ejectment,Possession,40.830857,-73.905191,3.0,16.0,145.0,2004227.0,2.026100e+09,Claremont-Bathgate
1,B047517/19,409031,4014 CARPENTER AVENUE,4B,11/16/22,Richard,McCoy,Residential,BRONX,10466,Not an Ejectment,Possession,40.889878,-73.862686,12.0,12.0,408.0,2063060.0,2.048280e+09,Williamsbridge-Olinville
2,15068/17,334442,655 EAST 224TH STREET,1,9/29/17,Thomas,Bia,Residential,BRONX,10467,Not an Ejectment,Possession,40.887599,-73.862391,12.0,12.0,394.0,2062985.0,2.048260e+09,Williamsbridge-Olinville
3,58273/18,25388,1551 DEAN STREET,1ST FLOOR,7/12/18,Gary,Rose,Residential,BROOKLYN,11213,Not an Ejectment,Possession,40.676166,-73.936661,8.0,36.0,311.0,3388499.0,3.013400e+09,Crown Heights North
4,14866/19A,97278,718 PENFIELD STREET,2-F,10/24/19,Justin,Grossman,Residential,BRONX,10470,Not an Ejectment,Possession,40.904888,-73.849089,12.0,11.0,442.0,2071873.0,2.051130e+09,Woodlawn-Wakefield


In [8]:
# counting total evictions by zip code
evictions.groupby(['Eviction Zip'])['Eviction Zip'].count()

Eviction Zip
0          3
1000       1
10000      1
10001    385
10002    372
        ... 
11693    108
11694    147
11698      1
11753      1
12221      1
Name: Eviction Zip, Length: 227, dtype: int64